In [ ]:
# Sapflow Analysis iPython Notebook - Jesse Hahm - December 2015
## Import required libraries

import matplotlib.pyplot as plt
#import mpld3
import pandas as pd
import glob
from IPython.core.display import HTML
import numpy as np
import scipy as sp
import functools
%matplotlib qt
import databaseQuery
import readSapflow

In [ ]:
## Read in the data as a pandas dataframe
# Files should be .csv format in \Data\ subdirectory

allFiles = ['RivSouth', 'RivNorth', 'SageSouth', 'SageNorth']

#Return pandas dataframe with all the data, datetime index, cleared of logger-statistics
#toggle saveCSV True/False to save a csv of the compiled data

sapflow = readSapflow.readCSV(allFiles, saveCSV = False)


In [ ]:

#upside down guy
sapflow['SVP87022']=-sapflow['SVP87022']
sapflow['SVP87022.1']=-sapflow['SVP87022.1']

In [ ]:
## Read in the metadata as a pandas dataframe
metadata = pd.read_csv('Data\sapflowMetadata.csv',sep=',')
metadata.tail()
#Read in zero flow times (found with script ZeroFlowTimes.py)
rivZero = pd.read_csv('AngeloMeadowZeroFlowTimes.csv', header=None, parse_dates={"datetime" : [0]})
sageZero = pd.read_csv('SagehornZeroFlowTimes.csv', header=None, parse_dates={"datetime" : [0]})
rivZero.set_index('datetime', inplace=True)
sageZero.set_index('datetime', inplace=True)

#Create dictionaries between sensor label and metadata info; sensor label = column headers
Location = metadata.set_index('CurrentSVP')['Location'].to_dict()
Slope = metadata.set_index('CurrentSVP')['Slope'].to_dict()
TreeID = metadata.set_index('CurrentSVP')['Tree ID'].to_dict()
Species = metadata.set_index('CurrentSVP')['Species'].to_dict()
Display = metadata.set_index('CurrentSVP')['Display'].to_dict()
#Sensors =  list(metadata('CurrentSVP').values())


In [ ]:
#only one depth for now
for sensor in sapflow.columns:
    if not sensor in Location:   
        sapflow.drop(sensor, axis=1, inplace=True)


In [ ]:
## Set date-time range of interest
startDateTime = pd.to_datetime('2015-08-1 00:00:00')
stopDateTime = pd.to_datetime('2016-02-15 00:00:00')
sapflow = sapflow[(sapflow.index > startDateTime) 
                 &(sapflow.index < stopDateTime)]
print sapflow.shape


# ## Plot the data

# nrows = len(sapflow.columns)

# f, axarr = plt.subplots(nrows, sharex=True, figsize=(20, nrows*2))

# for n in range(nrows):
#     axarr[n].plot(sapflow.index, sapflow.iloc[:,n], 
#                   label = sapflow.iloc[:,n].name
#                   + ', Location: ' + Location[sapflow.iloc[:,n].name]
#                   + ', Slope: ' + Slope[sapflow.iloc[:,n].name]
#                   + ', Species: ' + Species[sapflow.iloc[:,n].name]
#                   + ', TreeID: ' + TreeID[sapflow.iloc[:,n].name])
#     axarr[n].legend(bbox_to_anchor=(1, 1.2))
#  #axarr[0].set_ylabel("Sapflow velocity [cm / hr]")
# plt.show()
    



In [ ]:
#Get times when likely zero sap flow at both sites
sapflowZerosRiv = sapflow.join(rivZero, how='inner')
sapflowZerosSage = sapflow.join(sageZero, how='inner')

#Get median value for each sensor 
medianZeroRiv = sapflowZerosRiv.median().to_dict()
medianZeroSage = sapflowZerosSage.median().to_dict()

#Single dictionary of median zero sap flow value by site
medianZero = {}
for column in sapflow:
    if Location[column] == 'Rivendell':
        medianZero[column] = medianZeroRiv[column]
    if Location[column] == 'Sagehorn':
        medianZero[column] = medianZeroSage[column]


In [ ]:
#procedure outlined in matlab code by 
#P Link, who followed excel worksheets by A Ambrose and Burgess et al Tree Phys. 2001
import numpy as np
# parameters
k = 2.5e-3; # thermal diffusivity of fresh wood, cm^2/s, from Burgess 2001 - preliminary estimate, should be refined with measurements
t = 80; # 80 seconds measurement time, from p.591 in Burgess 2001
xnom = 0.6; # nominal probe spacing, in cm

def correctionEq(zeroValue):
    lnv1v2 = zeroValue*xnom/k/3600
    x1 = np.sqrt(xnom**2-4*k*t*lnv1v2)
    x2 = -np.sqrt(4*k*t*lnv1v2+xnom**2)
    vrange = np.linspace(0,20,200)
    lnv1v2 = vrange*xnom/3600/k
    veloc1s = (4*k*t*lnv1v2-(-xnom)**2+x1**2)*3600/(2*t*(x1-(-xnom)))
    veloc2s = (4*k*t*lnv1v2-(x2)**2+xnom**2)*3600/(2*t*(xnom-x2))
    vrange_corr = (veloc1s+veloc2s)/2
    p = np.polyfit(vrange,vrange_corr,1)
    slope = p[0]
    intercept = p[1]
    return slope, intercept

In [ ]:
#LEVEL 4A = derived quantity: corrected for probe misplacement
level4a = pd.DataFrame()
slopes = {}
intercepts = {}
for column in sapflow:
    slope, intercept = correctionEq(medianZero[column])
    slopes[column] = slope
    intercepts[column] = intercept
    level4a[column] = sapflow[column]*slope + intercept

In [ ]:
# %matplotlib inline
# plotData = level4a.copy()
# %matplotlib inline
# nrows = len(plotData.columns)
# f, axarr = plt.subplots(nrows, sharex=True, figsize=(20, nrows*2))

# for n in range(nrows):
#     if Display[plotData.iloc[:,n].name] == 'yes':
#         axarr[n].plot(plotData.index, plotData.iloc[:,n], 
#                       label = plotData.iloc[:,n].name
#                       + ', Location: ' + Location[plotData.iloc[:,n].name]
#                       + ', Slope: ' + Slope[plotData.iloc[:,n].name]
#                       + ', Species: ' + Species[plotData.iloc[:,n].name]
#                       + ', TreeID: ' + TreeID[plotData.iloc[:,n].name])
#         axarr[n].legend(bbox_to_anchor=(1, 1.2))
#  #axarr[0].set_ylabel("Sapflow velocity [cm / hr]")
# plt.show()
    


In [ ]:
#LEVEL 4B = remove values outside of acceptable ranges
level4b = level4a[(level4a > -5) &
                  (level4a < 50)]


# plotData = level4b.copy()
# %matplotlib inline
# nrows = len(plotData.columns)
# f, axarr = plt.subplots(nrows, sharex=True, figsize=(20, nrows*2))

# for n in range(nrows):
#     axarr[n].plot(plotData.index, plotData.iloc[:,n], 
#                   label = plotData.iloc[:,n].name
#                   + ', Location: ' + Location[plotData.iloc[:,n].name]
#                   + ', Slope: ' + Slope[plotData.iloc[:,n].name]
#                   + ', Species: ' + Species[plotData.iloc[:,n].name]
#                   + ', TreeID: ' + TreeID[plotData.iloc[:,n].name])
#     axarr[n].legend(bbox_to_anchor=(1, 1.2))
#  #axarr[0].set_ylabel("Sapflow velocity [cm / hr]")
# plt.show()
    


In [ ]:
## Import from mySQL database precipitation, solar radiation for similar time period
#Convert datetime stamps to strings for the sql query
dateStart = str(startDateTime.year) + '-' + str(startDateTime.month) + '-' + str(startDateTime.day)
dateStop =  str(stopDateTime.year) + '-' + str(stopDateTime.month) + '-' + str(stopDateTime.day)

#here we make use of external .py file with odmquery function (thanks Collin!) to get data from the database
#1672 = cumulative WY precip, in mm
#1785 = Rainfall mm TB4 WSAM RWS_Rain_TB4_Tot2015-03-26 (ongoing)Rainfall mmAngelo Meadow WSwsam0 to 700TB4mm-L Rain Gage
#1674	Solar Radiation Net W/m^2 Raw WSAM angelo meadow

#1687	Well 5 Water Level m
#1688	Well 6 Water Level m
#1689	Well 7 Water Level m
well5 = databaseQuery.odmquery(dateStart, dateStop, '1687', True)
well5.rename(columns = {'DataValue': 'well5'})
well6 = databaseQuery.odmquery(dateStart, dateStop, '1688', True)
well6.rename(columns = {'DataValue': 'well6'})
well7 = databaseQuery.odmquery(dateStart, dateStop, '1689', True)
well7.rename(columns = {'DataValue': 'well7'})

rain = databaseQuery.odmquery(dateStart, dateStop, '1785', True)
sun = databaseQuery.odmquery(dateStart, dateStop, '1674', True)
#Resample to daily rainfall
dailyRainfall = rain.resample('D', how=lambda x: x.sum())
maxSun = sun.resample('D', how = lambda x:x.max())

In [ ]:
## Trim outliers from data series by percentile cutoff
lowerPercentile = 0.025
upperPercentile = 0.99

sapflowLower = level4b.quantile(lowerPercentile)
sapflowUpper = level4b.quantile(upperPercentile)
# Level 4c = trim outliers
level4c = level4b [(level4b > sapflowLower) 
                 & (level4b < sapflowUpper)]

    



# plotData = level4c.copy()
# %matplotlib inline
# nrows = len(plotData.columns)
# f, axarr = plt.subplots(nrows, sharex=True, figsize=(20, nrows*2))

# for n in range(nrows):
#     if Display[plotData.iloc[:,n].name] == 'yes':
#         axarr[n].plot(plotData.index, plotData.iloc[:,n], 
#                       label = plotData.iloc[:,n].name
#                       + ', Location: ' + Location[plotData.iloc[:,n].name]
#                       + ', Slope: ' + Slope[plotData.iloc[:,n].name]
#                       + ', Species: ' + Species[plotData.iloc[:,n].name]
#                       + ', TreeID: ' + TreeID[plotData.iloc[:,n].name])
#         axarr[n].legend(bbox_to_anchor=(1, 1.2))
#  #axarr[0].set_ylabel("Sapflow velocity [cm / hr]")
# plt.show()
    


In [ ]:
## Normalize values: LEVEL 4D
sapflowMin = level4c.apply(np.min)

#center to zero
centered = level4c-sapflowMin


#Divide by maximum
sapflowMax = centered.apply(np.max)

level4d = centered.divide(sapflowMax)

# plotData = level4d.copy()
# %matplotlib inline
# nrows = len(plotData.columns)
# f, axarr = plt.subplots(nrows, sharex=True, figsize=(20, nrows*2))

# for n in range(nrows):
#     axarr[n].plot(plotData.index, plotData.iloc[:,n], 
#                   label = plotData.iloc[:,n].name
#                   + ', Location: ' + Location[plotData.iloc[:,n].name]
#                   + ', Slope: ' + Slope[plotData.iloc[:,n].name]
#                   + ', Species: ' + Species[plotData.iloc[:,n].name]
#                   + ', TreeID: ' + TreeID[plotData.iloc[:,n].name])
#     axarr[n].legend(bbox_to_anchor=(1, 1.2))
#  #axarr[0].set_ylabel("Sapflow velocity [cm / hr]")
# plt.show()
    




# ## Plot the data
# %matplotlib qt
# Plot = plotData.plot(alpha=.2)
# Plot.set_ylabel("Sapflow velocity (normalized to max)")
# for item in Plot.get_xticklabels():
#     item.set_rotation(-45)
# Plot.legend(bbox_to_anchor=(1.5, 1.5))


In [ ]:
# ## Plot daily maxima
# daily = plotData.resample('D', how=lambda x: x.max())

# Plot = daily.plot(alpha=0.1)
# Plot.set_ylabel("Daily maximum sapflow velocity (normalized to max)")
# for item in Plot.get_xticklabels():
#     item.set_rotation(-45)
# Plot.legend(bbox_to_anchor=(1.5, 1.5))

# #moving window average
# dailyWindow = pd.rolling_mean(daily, window=5).shift(-2)
# ## Plot the data
# %matplotlib qt
# Plot = dailyWindow.plot()
# Plot.set_ylabel("Sapflow velocity (normalized to max), five day moving window")
# for item in Plot.get_xticklabels():
#     item.set_rotation(-45)
# Plot.legend(bbox_to_anchor=(1.5, 1.5))
# #mpld3.display()

In [ ]:
plotData = level4d.copy()
#plotData = plotData.resample('24H', how=lambda x: x.max(), base=6)
#plotData = plotData.resample('24H', base=6)
#plotData = plotData.resample('1M', base=.5)

%matplotlib qt
speciesList = list(set(Species.values()))
nrows = len(speciesList)
f, axarr = plt.subplots(nrows+1, sharex=True, figsize=(20, 15))

for column in plotData:
    if Display[column] == 'yes':
        if Location[column] == 'Sagehorn':
            ls = '--'
        else:
            ls = '-'
        if Slope[column] == 'South':
            c = 'r'
        else:
            c = 'b'

        axarr[speciesList.index(Species[column])].plot(
                      plotData.index, 
                      plotData[column], 
                      color = c,
                      linestyle = ls)
        axarr[speciesList.index(Species[column])].legend(bbox_to_anchor=(1, 1.2))
#- pd.Timedelta(15, 'd'))
    
axarr[-1].plot(dailyRainfall.index, dailyRainfall['DataValue'], color='b')
axarr[-1].set_ylabel('Daily Rainfall [mm]')
axarr[-1].set_title('Solid=Rivendell; Dashed=Sagehorn; Red=South slope; Blue=North slope')
ax2 = axarr[-1].twinx()
ax2.plot(sun.index, sun['DataValue'], color='y')
ax2.plot(rain.index, rain['DataValue'].cumsum(), color = 'b')
ax2.set_ylabel('Radiation [W m$^{-2}$]; Cumulative Rain [mm]')
ax2.set_ylim([0,1500])

for n in range(nrows): 
    axarr[n].set_title(speciesList[n])
    
for n in range(nrows+1): 
#     axarr[n].yaxis.grid(color='gray', linestyle='dashed')
    axarr[n].xaxis.grid(color='gray', linestyle='dashed')
    
    
#axarr[2].set_ylabel('Individual Sensor Normalized Maximum Daily Sapflow Velocity')
axarr[2].set_ylabel('Normalized Sapflow Velocity')
axarr[-1].set_xlim(['2016-01-01 06:00:00','2016-01-31 06:00:00'])
f.suptitle('Summer 24H', fontsize=20)
plt.show()
#plt.savefig('DailyMax.pdf', pad_inches=0.1, frameon=None)

In [ ]:
## Plot the data

# Plot = dailyRainfall.plot()
# Plot.set_ylabel("DailyRainfall [mm]")
# for item in Plot.get_xticklabels():
#     item.set_rotation(-45)
# Plot.legend(bbox_to_anchor=(1.5, 1.5))


# Plot = dailyWindow.plot()
# Plot.set_ylabel("Sapflow velocity [Normalized], 5 day running maximum")
# for item in Plot.get_xticklabels():
#     item.set_rotation(-45)
# Plot.legend(bbox_to_anchor=(1.5, 1.5))


In [ ]:
level4c.tail()

In [ ]:
daily = level4d.resample(rule='24H', how='mean', closed='left', label='left', base=6)

In [ ]:
daynite = level4d.resample(rule='12H', how='mean', closed='left', label='left', base=6)

In [ ]:
weekly = level4d.resample(rule='1W', how='mean', closed='left', label='left')

In [ ]:
monthly = level4d.resample(rule='1M', how='mean', closed='left', label='left')

In [ ]:
grouper = pd.TimeGrouper('24H', base=6)

In [ ]:
dailyNorm = level4d.groupby(grouper).transform(lambda x: x/x.sum())

In [ ]:
dailyNorm.plot()

Questions:

What percentage of transpiration occurs at night throughout the year?

How does this vary as a function of mean night-time relative humidity and temperature?

What contributes to declines in mean transpiration more: decreases in length of day or rainy/sunless days?


In [ ]:
well5.plot()

In [ ]:
# %% WELL EXAM
plotData = level4d.copy()
#plotData = plotData.join(sun, how='outer')
plotStart = '2015-08-20'
plotStop = '2015-08-24'
plotData = plotData[(plotData.index > plotStart) &
                    (plotData.index < plotStop)]
plotSun = sun[(sun.index > plotStart) &
                    (sun.index < plotStop)]
plotWell5 = well5[(well5.index > plotStart) &
                    (well5.index < plotStop)]
plotWell6 = well6[(well6.index > plotStart) &
                    (well6.index < plotStop)]
plotWell7 = well7[(well7.index > plotStart) &
                    (well7.index < plotStop)]


%matplotlib qt
speciesList = list(set(Species.values()))
nrows = len(speciesList)
f, axarr = plt.subplots(nrows+1, sharex=True, figsize=(5, nrows*2))

for column in plotData:
    if Display[column] == 'yes':
        if Location[column] == 'Rivendell':
            ls = '-'
            if Slope[column] == 'North':
                c = 'b'


                axarr[speciesList.index(Species[column])].plot(
                              plotData.index, 
                              plotData[column], 
                              color = c,
                              linestyle = ls)
                axarr[speciesList.index(Species[column])].legend(bbox_to_anchor=(1, 1.2))



axarr[-1].plot(plotSun.index, plotSun['DataValue'], color='y')
axarr[-1].set_title('North Slope Rivendell Trees')
axarr[-1].set_ylabel('Radiation [W m^-2]')


for n in range(nrows): 
    axarr[n].set_title(speciesList[n])
    
for n in range(nrows+1): 
#     axarr[n].yaxis.grid(color='gray', linestyle='dashed')
    axarr[n].xaxis.grid(color='gray', linestyle='dashed')
    
#axarr[1].plot(plotWell5.index, plotWell5['DataValue']-plotWell5['DataValue'][0], label='5')
#axarr[1].plot(plotWell6.index, plotWell6['DataValue']-plotWell6['DataValue'][0], label='6')
axarr[1].plot(plotWell7.index, plotWell7['DataValue'], label='7')
axarr[1].legend(bbox_to_anchor=(1.1, 1.2))
axarr[1].set_title('Wells: August 10-20, 2015')

axarr[2].set_ylabel('Individual Sensor Normalized Maximum Daily Sapflow Velocity')
plt.show()


In [ ]:
import datetime
from astral import Astral
city_name = 'San Francisco'
a = Astral()
a.solar_depression = 'civil'
city = a[city_name]

In [ ]:
def sunrise(timestamp1):
    sr = city.sun(date = timestamp1, local=True)['sunrise']
    sr.replace(tzinfo=None)
    return sr
def sunset(timestamp1):
    ss = city.sun(date = timestamp1, local=True)['sunset']
    ss.replace(tzinfo=None)
    return ss

In [ ]:
daytime = level4d.copy()
for ts in daytime.index:
    if ~((ts.replace(tzinfo=None) > sunrise(ts).replace(tzinfo=None)) &
    (ts.replace(tzinfo=None) < sunset(ts).replace(tzinfo=None))):
        #print ts
        daytime.drop(ts, inplace=True, axis = 0)
